In [75]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import urllib.parse
from datetime import datetime
import pytz
import requests
import numpy as np

# Data Extraction

In [77]:
query1 = "SELECT userid as user_id, productid as product_id, productname as product_name, producttype as buy_type, adddate as add_date, effectivedate as effective_date, overduedate as overdue_date, lesson, unitprice as unit_price, productmoney as product_money FROM ehailuo_product_buy_glv"
order = pd.read_sql(query1, engine)
order.tail()

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money
231204,16499394.0,17.0,海螺币,3185.0,1.690463e+09,NaN,NaN,0.0,0.0,0.0
231205,16517179.0,17.0,海螺币,3185.0,1.690466e+09,NaN,NaN,0.0,0.0,0.0
231206,16581580.0,17.0,海螺币,3185.0,1.690466e+09,NaN,NaN,0.0,0.0,0.0
231207,16562491.0,41.0,取消约课卡,1151.0,1.690467e+09,1.690467e+09,1.690467e+09,2.0,0.0,0.0
231208,16583135.0,26.0,体验卡,749.0,1.690468e+09,1.690468e+09,1.759501e+09,0.0,0.0,0.0


In [78]:
query2 = "SELECT attriid as attri_id, attriname as attri_name, attriname_en as attri_name_en, pid FROM ehailuo_attribute_info_glv"
attribute = pd.read_sql(query2, engine)
attribute

,attri_id,attri_name,attri_name_en,pid
0,1,小学生,Elementary Students,0.0
1,2,中学生,Junior/Senior Students,0.0
2,3,大学生,Undergraduates,0.0
3,4,销售/客服/采购,Sales/Customer Service/Purchase,0.0
4,5,IT/通信/电子,IT/Communication/E-commerce,0.0
...,...,...,...,...
2752,3187,抖音直播,None,3172.0
2753,3188,CR30天停课学员回访,,NaN
2754,3189,青少儿应试,Young Learners Test Preparation,568.0
2755,3190,小红书投放,None,3172.0


In [79]:
query3 = "SELECT classorderid as class_order_id,classid as class_id, studentid as user_id, classorderstatus as class_order_status, paytype as pay_type, ordertime as order_time FROM ehailuo_class_orderlist_glv"
class_reservation = pd.read_sql(query3, engine)
class_reservation

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time
0,1,39,401,3,1,1396663200
1,2,52,402,1,1,1396514278
2,3,50,401,0,1,1396514983
3,4,44,401,2,1,1396515856
4,5,54,401,1,1,1396515925
...,...,...,...,...,...,...
1359719,1363903,2935928,16557469,1,25,1690472829
1359720,1363904,2935960,141620,1,21,1690473881
1359721,1363905,2884172,16576479,1,25,1690473887
1359722,1363906,2933018,16568079,1,25,1690475605


In [80]:
query4 = "SELECT classid as class_id, teacherid as teacher_id, teachertype as teacher_type, starttime as start_time, adddate as add_date,wday as w_day FROM ehailuo_class_schedule_glv"
class_detail = pd.read_sql(query4, engine)
class_detail

,class_id,teacher_id,teacher_type,start_time,add_date,w_day
0,39,402,1,1396663200,1395906274,6
1,40,402,1,1396711800,1395906274,6
2,41,406,2,1396146600,1396175299,0
3,42,406,2,1396175400,1396175299,0
4,43,406,2,1396670400,1396175339,6
...,...,...,...,...,...,...
2926875,2935995,2064,2,1690538400,1690475457,5
2926876,2935996,838,5,1690959600,1690476036,3
2926877,2935997,838,5,1690956000,1690476038,3
2926878,2935998,838,5,1691046000,1690476041,4


In [81]:
query5 = "SELECT classorderid as class_order_id, startattr1 as issue_type_1, startattr2 as issue_type_2, content as issue_content FROM ehailuo_feedback"
class_feedback = pd.read_sql(query5, engine)
class_feedback

,class_order_id,issue_type_1,issue_type_2,issue_content
0,110384,0,0,老师缺席课程
1,110384,0,0,老师缺席课程 已经扣除老师两倍工资
2,110707,0,0,Pat 老师缺席\r\n老师9点和11点都有课 按理不会缺席10点的课程 但是因为学员是临时...
3,113216,925,0,由于老师网络出现问题，QQ和skype都无法进行正常上课，之后就临时换了平台上课，但是后期还...
4,113015,925,0,学员：马雷\r\n账号：13663002657\r\n上课时间：2.10 22:00-22:...
...,...,...,...,...
33125,1360868,0,0,未及时填写课后评价
33126,1358795,0,0,未及时填写课后评价
33127,1362043,0,0,返还学员本节课时，老师按设备问题处理
33128,1360403,0,0,老师设备问题，少上了10分钟，申请补偿。


In [82]:
query6 = "SELECT order_id as class_order_id, type_id as issue_type FROM ehailuo_feedback_auto"
class_feedback_auto = pd.read_sql(query6, engine)
class_feedback_auto

,class_order_id,issue_type
0,1329397,3175
1,1328083,3183
2,1329255,3176
3,1327062,3175
4,1327039,3175
...,...,...
1134,1363266,3179
1135,1363018,3175
1136,1363335,3179
1137,1363169,3175


In [83]:
query7 = "SELECT classorderid as class_order_id, totalpoint as total_point, material FROM ehailuo_class_eval_glv"
class_comment = pd.read_sql(query7, engine)
class_comment = class_comment.drop_duplicates()
class_comment

,class_order_id,total_point,material
0,10.0,5,5
1,18.0,1,4
2,47.0,5,3
3,205.0,5,5
4,216.0,5,5
...,...,...,...
36778,1337832.0,5,5
36779,1363345.0,5,4
36780,1361891.0,5,5
36781,1363018.0,4,5


In [84]:
query8 = "SELECT userid as user_id, crid as cr_id FROM ehailuo_member"
cr = pd.read_sql(query8, engine)
cr

,user_id,cr_id
0,100001,0.0
1,100002,0.0
2,100100,0.0
3,113040,156.0
4,113041,0.0
...,...,...
237804,16583131,0.0
237805,16583132,0.0
237806,16583133,0.0
237807,16583134,0.0


In [85]:
member_demographics = pd.read_csv('member_demographics.csv')
member_demographics

,user_id,age,gender,job,after_policy
0,131638,420.0,1,0,0
1,131639,417.0,0,0,0
2,131640,413.0,2,0,0
3,131641,416.0,0,0,0
4,131642,420.0,0,0,0
...,...,...,...,...,...
158085,16579599,412.0,2,16,1
158086,16579600,417.0,0,16,1
158087,16579601,412.0,2,1,1
158088,16579602,419.0,0,16,1


In [86]:
member_analysis_list = member_demographics['user_id'].tolist()

# Customer Repurchase

In [87]:
official_order = order[order['product_id'].isin([21,23,25,27,28,32,33,34,35,38,42,62,64,66])]

In [88]:
buy_type_list = official_order['buy_type'].unique().tolist()
attribute_buy_type = attribute[attribute['attri_id'].isin(buy_type_list)]

In [89]:
buy_type_new = attribute_buy_type[attribute_buy_type['attri_name'].isin(['升级充值','直接付费','官网自己充值','新签','新签升级',
                                                            '推荐升级','推荐直接付费','新签扩包'])]

In [90]:
order_new = official_order[official_order['buy_type'].isin(buy_type_new['attri_id'])]
order_new

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money
0,113040.0,21.0,菲律宾外教,105.0,NaN,1.412930e+09,1.415522e+09,16.0,0.0,0.0
1,113040.0,25.0,欧美外教次卡,105.0,NaN,1.412930e+09,1.415522e+09,14.0,0.0,0.0
3,113040.0,21.0,菲律宾外教,105.0,NaN,1.412742e+09,1.415334e+09,16.0,0.0,0.0
4,113040.0,25.0,欧美外教次卡,105.0,NaN,1.412742e+09,1.415334e+09,14.0,0.0,0.0
8,113040.0,21.0,菲律宾外教,105.0,1.412956e+09,1.412958e+09,1.415550e+09,16.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
231036,16582872.0,25.0,欧美外教次卡,710.0,1.690356e+09,NaN,NaN,100.0,179.0,17900.0
231045,16582701.0,25.0,欧美外教次卡,710.0,1.690358e+09,1.690360e+09,1.702802e+09,48.0,187.5,8999.0
231088,16582687.0,25.0,欧美外教次卡,710.0,1.690371e+09,NaN,NaN,50.0,180.0,8999.0
231144,16582197.0,25.0,欧美外教次卡,710.0,1.690442e+09,NaN,NaN,32.0,209.0,6688.0


In [91]:
order_new = order_new[order_new['product_money']!=0]
order_new = order_new[order_new['lesson']>=5]
order_new = order_new[order_new['user_id'].isin(member_analysis_list)]
order_new

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money
5749,131650.0,21.0,菲律宾外教次卡,105.0,1.451713e+09,1.452308e+09,1.460602e+09,32.0,70.1,2244.0
5750,131669.0,25.0,欧美外教次卡,105.0,1.451713e+09,1.455611e+09,1.468053e+09,48.0,149.4,7172.0
5770,131648.0,25.0,欧美外教次卡,105.0,1.451875e+09,1.452072e+09,1.454146e+09,8.0,143.0,1144.0
5845,131698.0,25.0,欧美外教次卡,105.0,1.451976e+09,1.451979e+09,1.464680e+09,49.0,146.0,7152.0
5849,131730.0,25.0,欧美外教次卡,105.0,1.451982e+09,1.452816e+09,1.456963e+09,16.0,138.9,2223.0
...,...,...,...,...,...,...,...,...,...,...
230144,16401660.0,25.0,欧美外教次卡,710.0,1.689651e+09,1.689689e+09,1.697983e+09,32.0,218.8,7000.0
230207,16559095.0,25.0,欧美外教次卡,710.0,1.689672e+09,1.689909e+09,1.707535e+09,68.0,196.7,13376.0
230399,16564940.0,25.0,欧美外教次卡,715.0,1.689842e+09,1.689940e+09,1.706528e+09,64.0,199.0,12736.0
230907,16561546.0,25.0,欧美外教次卡,710.0,1.690272e+09,1.690272e+09,1.703232e+09,50.0,180.0,8999.0


In [92]:
order_new['user_id'].value_counts()

157296.0      5
16510596.0    4
16495885.0    4
16571168.0    4
16499368.0    4
             ..
165886.0      1
172363.0      1
16397457.0    1
171978.0      1
16561497.0    1
Name: user_id, Length: 11829, dtype: int64

In [93]:
# userid who have mutiple new orders
new_order_per_user = order_new['user_id'].value_counts()
user_with_multiple_new_orders = new_order_per_user[new_order_per_user > 1].index.tolist()
print(len(user_with_multiple_new_orders))

423


In [94]:
order_new_sorted = order_new.sort_values(by='add_date', ascending=True)
order_new_first = order_new_sorted.drop_duplicates(subset='user_id', keep='first')
order_new_first

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money
5749,131650.0,21.0,菲律宾外教次卡,105.0,1.451713e+09,1.452308e+09,1.460602e+09,32.0,70.1,2244.0
5750,131669.0,25.0,欧美外教次卡,105.0,1.451713e+09,1.455611e+09,1.468053e+09,48.0,149.4,7172.0
5770,131648.0,25.0,欧美外教次卡,105.0,1.451875e+09,1.452072e+09,1.454146e+09,8.0,143.0,1144.0
5845,131698.0,25.0,欧美外教次卡,105.0,1.451976e+09,1.451979e+09,1.464680e+09,49.0,146.0,7152.0
5849,131730.0,25.0,欧美外教次卡,105.0,1.451982e+09,1.452816e+09,1.456963e+09,16.0,138.9,2223.0
...,...,...,...,...,...,...,...,...,...,...
230144,16401660.0,25.0,欧美外教次卡,710.0,1.689651e+09,1.689689e+09,1.697983e+09,32.0,218.8,7000.0
230207,16559095.0,25.0,欧美外教次卡,710.0,1.689672e+09,1.689909e+09,1.707535e+09,68.0,196.7,13376.0
230399,16564940.0,25.0,欧美外教次卡,715.0,1.689842e+09,1.689940e+09,1.706528e+09,64.0,199.0,12736.0
230907,16561546.0,25.0,欧美外教次卡,710.0,1.690272e+09,1.690272e+09,1.703232e+09,50.0,180.0,8999.0


In [95]:
# Convert UNIX timestamps to datetime objects
order_new_first['add_date'] = pd.to_datetime(order_new_first['add_date'], unit='s')
order_new_first['effective_date'] = pd.to_datetime(order_new_first['effective_date'], unit='s')
order_new_first['overdue_date'] = pd.to_datetime(order_new_first['overdue_date'], unit='s')

# Set timezone to China timezone
china_tz = pytz.timezone('Asia/Shanghai')
order_new_first['add_date'] = order_new_first['add_date'].dt.tz_localize(pytz.utc).dt.tz_convert(china_tz)
order_new_first['effective_date'] = order_new_first['effective_date'].dt.tz_localize(pytz.utc).dt.tz_convert(china_tz)
order_new_first['overdue_date'] = order_new_first['overdue_date'].dt.tz_localize(pytz.utc).dt.tz_convert(china_tz)

# Extract yyyy-mm-dd format from datetime objects
order_new_first['add_date'] = order_new_first['add_date'].dt.strftime('%Y-%m-%d %H:%M')
order_new_first['effective_date'] = order_new_first['effective_date'].dt.strftime('%Y-%m-%d %H:%M')
order_new_first['overdue_date'] = order_new_first['overdue_date'].dt.strftime('%Y-%m-%d %H:%M')

order_new_first

/var/folders/b9/g48b_66n63n3rwlbb7dhxm0r0000gn/T/ipykernel_48836/3123429903.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_new_first['add_date'] = pd.to_datetime(order_new_first['add_date'], unit='s')
/var/folders/b9/g48b_66n63n3rwlbb7dhxm0r0000gn/T/ipykernel_48836/3123429903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_new_first['effective_date'] = pd.to_datetime(order_new_first['effective_date'], unit='s')
/var/folders/b9/g48b_66n63n3rwlbb7dhxm0r0000gn/T/ipykernel_48836/3123429903.

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money
5749,131650.0,21.0,菲律宾外教次卡,105.0,2016-01-02 13:29,2016-01-09 10:52,2016-04-14 10:52,32.0,70.1,2244.0
5750,131669.0,25.0,欧美外教次卡,105.0,2016-01-02 13:30,2016-02-16 16:29,2016-07-09 16:29,48.0,149.4,7172.0
5770,131648.0,25.0,欧美外教次卡,105.0,2016-01-04 10:29,2016-01-06 17:22,2016-01-30 17:22,8.0,143.0,1144.0
5845,131698.0,25.0,欧美外教次卡,105.0,2016-01-05 14:32,2016-01-05 15:36,2016-05-31 15:36,49.0,146.0,7152.0
5849,131730.0,25.0,欧美外教次卡,105.0,2016-01-05 16:18,2016-01-15 07:56,2016-03-03 07:56,16.0,138.9,2223.0
...,...,...,...,...,...,...,...,...,...,...
230144,16401660.0,25.0,欧美外教次卡,710.0,2023-07-18 11:30,2023-07-18 21:55,2023-10-22 21:55,32.0,218.8,7000.0
230207,16559095.0,25.0,欧美外教次卡,710.0,2023-07-18 17:19,2023-07-21 11:13,2024-02-10 11:13,68.0,196.7,13376.0
230399,16564940.0,25.0,欧美外教次卡,715.0,2023-07-20 16:32,2023-07-21 19:40,2024-01-29 19:40,64.0,199.0,12736.0
230907,16561546.0,25.0,欧美外教次卡,710.0,2023-07-25 15:57,2023-07-25 15:59,2023-12-22 15:59,50.0,180.0,8999.0


In [96]:
order_new_activated = order_new_first[~order_new_first['effective_date'].isnull()]
order_new_completed = order_new_activated[(order_new_activated['overdue_date'] < '2023-04-30') & (order_new_activated['overdue_date'] > '2014-01-01')]
order_new_completed = order_new_completed.drop(order_new_completed[order_new_completed['effective_date'] == order_new_completed['overdue_date']].index)
order_new_completed

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money
5749,131650.0,21.0,菲律宾外教次卡,105.0,2016-01-02 13:29,2016-01-09 10:52,2016-04-14 10:52,32.0,70.1,2244.0
5750,131669.0,25.0,欧美外教次卡,105.0,2016-01-02 13:30,2016-02-16 16:29,2016-07-09 16:29,48.0,149.4,7172.0
5770,131648.0,25.0,欧美外教次卡,105.0,2016-01-04 10:29,2016-01-06 17:22,2016-01-30 17:22,8.0,143.0,1144.0
5845,131698.0,25.0,欧美外教次卡,105.0,2016-01-05 14:32,2016-01-05 15:36,2016-05-31 15:36,49.0,146.0,7152.0
5849,131730.0,25.0,欧美外教次卡,105.0,2016-01-05 16:18,2016-01-15 07:56,2016-03-03 07:56,16.0,138.9,2223.0
...,...,...,...,...,...,...,...,...,...,...
215160,16575719.0,25.0,欧美外教次卡,715.0,2023-01-30 15:12,2023-01-30 15:13,2023-03-31 15:13,20.0,219.0,4380.0
216212,16576144.0,25.0,欧美外教次卡,710.0,2023-02-11 17:40,2023-02-14 10:20,2023-03-16 10:20,10.0,219.0,2190.0
217224,16576983.0,25.0,欧美外教次卡,710.0,2023-02-23 13:09,2023-02-24 18:38,2023-04-21 18:38,8.0,219.0,1752.0
217735,16577191.0,25.0,欧美外教次卡,710.0,2023-02-28 21:21,2023-03-01 17:21,2023-03-31 17:21,10.0,220.0,2200.0


In [97]:
member_repurchase_analysis = order_new_completed['user_id'].unique().tolist()

In [98]:
# Read the CSV file into a DataFrame
customer_reach = pd.read_csv('customer_reach.csv')
customer_reach

,user_id,cc_id,num_cc,channel,trial_money,trial_completed
0,16410781,0.0,0.0,1662.0,0.0,0
1,16448486,100.0,4.0,97.0,0.0,1
2,16531800,650.0,2.0,2045.0,0.0,1
3,16532204,100.0,2.0,97.0,0.0,1
4,16398016,197.0,0.0,510.0,0.0,0
...,...,...,...,...,...,...
158085,16558015,647.0,3.0,1662.0,0.0,0
158086,16569340,647.0,0.0,1662.0,0.0,0
158087,16578239,647.0,0.0,1662.0,0.0,0
158088,16578345,647.0,0.0,1662.0,0.0,0


In [99]:
customer_repurchase = customer_reach[customer_reach['user_id'].isin(member_repurchase_analysis)]
customer_repurchase = customer_repurchase[['user_id', 'channel']]
customer_repurchase

,user_id,channel
12158,133809,99.0
12899,168116,97.0
15096,16485247,99.0
15154,16531907,99.0
15228,148917,94.0
...,...,...
135966,16528592,99.0
136220,16554336,99.0
136314,16564800,99.0
145902,16501431,0.0


In [100]:
official_class = class_reservation[class_reservation['user_id'].isin(member_repurchase_analysis)]
official_class

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time
20309,20310,84149,131648,2,26,1451565133
20311,20312,93459,131650,2,26,1451568453
20388,20389,93114,131647,0,26,1451716499
20494,20495,92595,131685,0,26,1451812578
20583,20584,94483,131665,2,26,1451891452
...,...,...,...,...,...,...
1359714,1363898,2928008,146608,1,25,1690471038
1359715,1363899,2935812,146608,1,25,1690471136
1359718,1363902,2934702,16557469,1,25,1690472800
1359719,1363903,2935928,16557469,1,25,1690472829


In [101]:
official_class = official_class.merge(order_new_completed[['user_id','product_id','effective_date','overdue_date']],on='user_id',how='left')
official_class

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date
0,20310,84149,131648,2,26,1451565133,25.0,2016-01-06 17:22,2016-01-30 17:22
1,20312,93459,131650,2,26,1451568453,21.0,2016-01-09 10:52,2016-04-14 10:52
2,20389,93114,131647,0,26,1451716499,25.0,2016-01-22 14:08,2016-11-05 14:08
3,20495,92595,131685,0,26,1451812578,21.0,2016-01-27 13:33,2016-05-02 13:33
4,20584,94483,131665,2,26,1451891452,25.0,2016-01-08 11:09,2016-05-31 11:09
...,...,...,...,...,...,...,...,...,...
558399,1363898,2928008,146608,1,25,1690471038,25.0,2016-11-07 16:01,2017-11-26 16:01
558400,1363899,2935812,146608,1,25,1690471136,25.0,2016-11-07 16:01,2017-11-26 16:01
558401,1363902,2934702,16557469,1,25,1690472800,25.0,2021-12-06 19:36,2022-11-07 19:36
558402,1363903,2935928,16557469,1,25,1690472829,25.0,2021-12-06 19:36,2022-11-07 19:36


In [102]:
# Convert UNIX timestamps to datetime objects
official_class['order_time'] = pd.to_datetime(official_class['order_time'], unit='s')

# Set timezone to China timezone
china_tz = pytz.timezone('Asia/Shanghai')
official_class['order_time'] = official_class['order_time'].dt.tz_localize(pytz.utc).dt.tz_convert(china_tz)

# Extract yyyy-mm-dd format from datetime objects
official_class['order_time'] = official_class['order_time'].dt.strftime('%Y-%m-%d %H:%M')

official_class

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date
0,20310,84149,131648,2,26,2015-12-31 20:32,25.0,2016-01-06 17:22,2016-01-30 17:22
1,20312,93459,131650,2,26,2015-12-31 21:27,21.0,2016-01-09 10:52,2016-04-14 10:52
2,20389,93114,131647,0,26,2016-01-02 14:34,25.0,2016-01-22 14:08,2016-11-05 14:08
3,20495,92595,131685,0,26,2016-01-03 17:16,21.0,2016-01-27 13:33,2016-05-02 13:33
4,20584,94483,131665,2,26,2016-01-04 15:10,25.0,2016-01-08 11:09,2016-05-31 11:09
...,...,...,...,...,...,...,...,...,...
558399,1363898,2928008,146608,1,25,2023-07-27 23:17,25.0,2016-11-07 16:01,2017-11-26 16:01
558400,1363899,2935812,146608,1,25,2023-07-27 23:18,25.0,2016-11-07 16:01,2017-11-26 16:01
558401,1363902,2934702,16557469,1,25,2023-07-27 23:46,25.0,2021-12-06 19:36,2022-11-07 19:36
558402,1363903,2935928,16557469,1,25,2023-07-27 23:47,25.0,2021-12-06 19:36,2022-11-07 19:36


In [103]:
official_class = official_class[(official_class['order_time'] >= official_class['effective_date'])]
official_class = official_class[(official_class['product_id'] == official_class['pay_type'])]
official_class

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date
10,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36
11,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36
12,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36
17,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27
20,20892,95252,131733,2,25,2016-01-07 10:21,25.0,2016-01-07 10:19,2016-03-07 10:19
...,...,...,...,...,...,...,...,...,...
558398,1363896,2926414,16450122,1,21,2023-07-27 23:06,21.0,2019-06-23 10:56,2020-02-03 10:56
558399,1363898,2928008,146608,1,25,2023-07-27 23:17,25.0,2016-11-07 16:01,2017-11-26 16:01
558400,1363899,2935812,146608,1,25,2023-07-27 23:18,25.0,2016-11-07 16:01,2017-11-26 16:01
558401,1363902,2934702,16557469,1,25,2023-07-27 23:46,25.0,2021-12-06 19:36,2022-11-07 19:36


#### if_completed

In [104]:
official_class_completed = official_class[official_class['class_order_status']==2]
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date
10,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36
11,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36
12,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36
17,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27
20,20892,95252,131733,2,25,2016-01-07 10:21,25.0,2016-01-07 10:19,2016-03-07 10:19
...,...,...,...,...,...,...,...,...,...
558221,1363283,2934713,16553205,2,21,2023-07-26 16:31,21.0,2021-08-01 11:56,2022-03-17 11:56
558226,1363303,2921012,16502780,2,25,2023-07-26 16:54,25.0,2021-02-06 21:59,2021-11-21 21:59
558256,1363399,2932844,16402387,2,25,2023-07-26 20:36,25.0,2018-03-18 22:07,2018-09-26 22:07
558265,1363436,2924750,141451,2,25,2023-07-26 22:15,25.0,2016-09-04 15:43,2017-06-10 15:43


In [105]:
official_class_completed = official_class_completed.merge(class_detail,on='class_id',how='left')
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,teacher_type,start_time,add_date,w_day
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,1452049200,1451871269,3
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,1452052800,1451871299,3
2,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,2,1452160800,1450786493,4
3,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,1452132000,1452080945,4
4,20892,95252,131733,2,25,2016-01-07 10:21,25.0,2016-01-07 10:19,2016-03-07 10:19,231,2,1452380400,1451933612,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401832,1363283,2934713,16553205,2,21,2023-07-26 16:31,21.0,2021-08-01 11:56,2022-03-17 11:56,1824,1,1690380000,1690358974,3
401833,1363303,2921012,16502780,2,25,2023-07-26 16:54,25.0,2021-02-06 21:59,2021-11-21 21:59,1460,2,1690444800,1689052492,4
401834,1363399,2932844,16402387,2,25,2023-07-26 20:36,25.0,2018-03-18 22:07,2018-09-26 22:07,2059,2,1690426800,1690195291,4
401835,1363436,2924750,141451,2,25,2023-07-26 22:15,25.0,2016-09-04 15:43,2017-06-10 15:43,2094,2,1690434000,1689488092,4


In [106]:
# Convert UNIX timestamps to datetime objects
official_class_completed['start_time'] = pd.to_datetime(official_class_completed['start_time'], unit='s')

# Set timezone to China timezone
china_tz = pytz.timezone('Asia/Shanghai')
official_class_completed['start_time'] = official_class_completed['start_time'].dt.tz_localize(pytz.utc).dt.tz_convert(china_tz)

# Extract yyyy-mm-dd format from datetime objects
official_class_completed['start_time'] = official_class_completed['start_time'].dt.strftime('%Y-%m-%d %H:%M')

official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,teacher_type,start_time,add_date,w_day
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 11:00,1451871269,3
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 12:00,1451871299,3
2,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,2,2016-01-07 18:00,1450786493,4
3,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 10:00,1452080945,4
4,20892,95252,131733,2,25,2016-01-07 10:21,25.0,2016-01-07 10:19,2016-03-07 10:19,231,2,2016-01-10 07:00,1451933612,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401832,1363283,2934713,16553205,2,21,2023-07-26 16:31,21.0,2021-08-01 11:56,2022-03-17 11:56,1824,1,2023-07-26 22:00,1690358974,3
401833,1363303,2921012,16502780,2,25,2023-07-26 16:54,25.0,2021-02-06 21:59,2021-11-21 21:59,1460,2,2023-07-27 16:00,1689052492,4
401834,1363399,2932844,16402387,2,25,2023-07-26 20:36,25.0,2018-03-18 22:07,2018-09-26 22:07,2059,2,2023-07-27 11:00,1690195291,4
401835,1363436,2924750,141451,2,25,2023-07-26 22:15,25.0,2016-09-04 15:43,2017-06-10 15:43,2094,2,2023-07-27 13:00,1689488092,4


In [107]:
official_class_completed = official_class_completed[(official_class_completed['start_time'] <= official_class_completed['overdue_date'])]
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,teacher_type,start_time,add_date,w_day
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 11:00,1451871269,3
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 12:00,1451871299,3
2,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,2,2016-01-07 18:00,1450786493,4
3,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 10:00,1452080945,4
4,20892,95252,131733,2,25,2016-01-07 10:21,25.0,2016-01-07 10:19,2016-03-07 10:19,231,2,2016-01-10 07:00,1451933612,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396009,1327720,2792379,16572556,2,25,2023-04-20 21:08,25.0,2022-10-25 16:31,2023-04-29 16:31,1488,2,2023-04-22 18:00,1677379208,6
396041,1327815,2795507,16565530,2,25,2023-04-21 09:10,25.0,2022-05-27 16:51,2023-04-28 16:51,800,2,2023-04-26 16:00,1677654661,3
396099,1328121,2831959,16570863,2,25,2023-04-22 10:25,25.0,2022-09-15 15:53,2023-04-27 15:53,1040,5,2023-04-22 20:00,1681632035,6
396153,1328440,2834921,16567961,2,25,2023-04-23 07:25,25.0,2022-07-09 11:30,2023-04-29 11:30,827,2,2023-04-26 17:00,1681901585,3


In [108]:
official_class_completed = official_class_completed.sort_values(by='start_time', ascending=True)


official_class_completed['row_rank'] = official_class_completed.groupby('user_id').cumcount() + 1
official_class_completed = official_class_completed.merge(order_new_completed[['user_id','lesson']],on='user_id',how='left')
official_class_completed = official_class_completed[official_class_completed['row_rank'] <= official_class_completed['lesson']]
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,teacher_type,start_time,add_date,w_day,row_rank,lesson
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 11:00,1451871269,3,1,49.0
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 12:00,1451871299,3,2,49.0
2,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 10:00,1452080945,4,1,14.0
3,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,2,2016-01-07 18:00,1450786493,4,3,49.0
4,20935,95052,131732,2,32,2016-01-07 17:30,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 21:00,1451897903,4,2,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210929,1326792,2830267,16565435,2,25,2023-04-18 09:56,25.0,2022-05-08 12:12,2023-04-23 12:12,1898,2,2023-04-22 20:00,1681478224,6,48,50.0
210930,1328121,2831959,16570863,2,25,2023-04-22 10:25,25.0,2022-09-15 15:53,2023-04-27 15:53,1040,5,2023-04-22 20:00,1681632035,6,21,32.0
210931,1326632,2826472,16566901,2,23,2023-04-17 19:32,23.0,2022-06-14 10:42,2023-04-25 10:42,1419,5,2023-04-22 21:00,1681088884,6,105,105.0
210932,1327815,2795507,16565530,2,25,2023-04-21 09:10,25.0,2022-05-27 16:51,2023-04-28 16:51,800,2,2023-04-26 16:00,1677654661,3,29,48.0


In [109]:
official_class_completed['completed_time'] = np.where(official_class_completed['row_rank'] == official_class_completed['lesson'], official_class_completed['start_time'], np.nan)
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,teacher_type,start_time,add_date,w_day,row_rank,lesson,completed_time
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 11:00,1451871269,3,1,49.0,NaN
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 12:00,1451871299,3,2,49.0,NaN
2,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 10:00,1452080945,4,1,14.0,NaN
3,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,2,2016-01-07 18:00,1450786493,4,3,49.0,NaN
4,20935,95052,131732,2,32,2016-01-07 17:30,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 21:00,1451897903,4,2,14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210929,1326792,2830267,16565435,2,25,2023-04-18 09:56,25.0,2022-05-08 12:12,2023-04-23 12:12,1898,2,2023-04-22 20:00,1681478224,6,48,50.0,NaN
210930,1328121,2831959,16570863,2,25,2023-04-22 10:25,25.0,2022-09-15 15:53,2023-04-27 15:53,1040,5,2023-04-22 20:00,1681632035,6,21,32.0,NaN
210931,1326632,2826472,16566901,2,23,2023-04-17 19:32,23.0,2022-06-14 10:42,2023-04-25 10:42,1419,5,2023-04-22 21:00,1681088884,6,105,105.0,2023-04-22 21:00
210932,1327815,2795507,16565530,2,25,2023-04-21 09:10,25.0,2022-05-27 16:51,2023-04-28 16:51,800,2,2023-04-26 16:00,1677654661,3,29,48.0,NaN


In [110]:
completed_user = official_class_completed[~official_class_completed['completed_time'].isnull()]
completed_user = completed_user[['user_id','completed_time']]
completed_user

,user_id,completed_time
146,131974,2016-01-28 21:00
174,131733,2016-01-31 07:00
451,132178,2016-02-17 10:00
500,132320,2016-02-19 16:00
600,132005,2016-02-24 19:00
...,...,...
210759,16564241,2023-03-29 19:00
210861,16574704,2023-04-08 17:00
210872,16570367,2023-04-11 19:00
210900,16571822,2023-04-16 15:00


In [111]:
completed_user_list = completed_user['user_id'].tolist()
# customer_repurchase['if_completed'] = customer_repurchase['user_id'].isin(completed_user_list).astype(int)
customer_repurchase

,user_id,channel
12158,133809,99.0
12899,168116,97.0
15096,16485247,99.0
15154,16531907,99.0
15228,148917,94.0
...,...,...
135966,16528592,99.0
136220,16554336,99.0
136314,16564800,99.0
145902,16501431,0.0


In [112]:
order_repurchase_analysis = order_new_completed.merge (completed_user[['user_id','completed_time']],on='user_id',how='left')
order_repurchase_analysis['completed_time'] = order_repurchase_analysis['completed_time'].fillna(order_repurchase_analysis['overdue_date'])
order_repurchase_analysis

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money,completed_time
0,131650.0,21.0,菲律宾外教次卡,105.0,2016-01-02 13:29,2016-01-09 10:52,2016-04-14 10:52,32.0,70.1,2244.0,2016-04-14 10:52
1,131669.0,25.0,欧美外教次卡,105.0,2016-01-02 13:30,2016-02-16 16:29,2016-07-09 16:29,48.0,149.4,7172.0,2016-07-09 16:29
2,131648.0,25.0,欧美外教次卡,105.0,2016-01-04 10:29,2016-01-06 17:22,2016-01-30 17:22,8.0,143.0,1144.0,2016-01-30 17:22
3,131698.0,25.0,欧美外教次卡,105.0,2016-01-05 14:32,2016-01-05 15:36,2016-05-31 15:36,49.0,146.0,7152.0,2016-05-31 15:36
4,131730.0,25.0,欧美外教次卡,105.0,2016-01-05 16:18,2016-01-15 07:56,2016-03-03 07:56,16.0,138.9,2223.0,2016-03-03 07:56
...,...,...,...,...,...,...,...,...,...,...,...
10241,16575719.0,25.0,欧美外教次卡,715.0,2023-01-30 15:12,2023-01-30 15:13,2023-03-31 15:13,20.0,219.0,4380.0,2023-02-24 16:00
10242,16576144.0,25.0,欧美外教次卡,710.0,2023-02-11 17:40,2023-02-14 10:20,2023-03-16 10:20,10.0,219.0,2190.0,2023-03-09 19:00
10243,16576983.0,25.0,欧美外教次卡,710.0,2023-02-23 13:09,2023-02-24 18:38,2023-04-21 18:38,8.0,219.0,1752.0,2023-03-12 11:00
10244,16577191.0,25.0,欧美外教次卡,710.0,2023-02-28 21:21,2023-03-01 17:21,2023-03-31 17:21,10.0,220.0,2200.0,2023-03-31 17:21


In [113]:
order_repurchase_analysis['completed_time'] = pd.to_datetime(order_repurchase_analysis['completed_time'])
order_repurchase_analysis['effective_date'] = pd.to_datetime(order_repurchase_analysis['effective_date'])
order_repurchase_analysis['num_days_completed'] = (order_repurchase_analysis['completed_time'] - order_repurchase_analysis['effective_date']).dt.days

order_repurchase_analysis

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money,completed_time,num_days_completed
0,131650.0,21.0,菲律宾外教次卡,105.0,2016-01-02 13:29,2016-01-09 10:52:00,2016-04-14 10:52,32.0,70.1,2244.0,2016-04-14 10:52:00,96
1,131669.0,25.0,欧美外教次卡,105.0,2016-01-02 13:30,2016-02-16 16:29:00,2016-07-09 16:29,48.0,149.4,7172.0,2016-07-09 16:29:00,144
2,131648.0,25.0,欧美外教次卡,105.0,2016-01-04 10:29,2016-01-06 17:22:00,2016-01-30 17:22,8.0,143.0,1144.0,2016-01-30 17:22:00,24
3,131698.0,25.0,欧美外教次卡,105.0,2016-01-05 14:32,2016-01-05 15:36:00,2016-05-31 15:36,49.0,146.0,7152.0,2016-05-31 15:36:00,147
4,131730.0,25.0,欧美外教次卡,105.0,2016-01-05 16:18,2016-01-15 07:56:00,2016-03-03 07:56,16.0,138.9,2223.0,2016-03-03 07:56:00,48
...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16575719.0,25.0,欧美外教次卡,715.0,2023-01-30 15:12,2023-01-30 15:13:00,2023-03-31 15:13,20.0,219.0,4380.0,2023-02-24 16:00:00,25
10242,16576144.0,25.0,欧美外教次卡,710.0,2023-02-11 17:40,2023-02-14 10:20:00,2023-03-16 10:20,10.0,219.0,2190.0,2023-03-09 19:00:00,23
10243,16576983.0,25.0,欧美外教次卡,710.0,2023-02-23 13:09,2023-02-24 18:38:00,2023-04-21 18:38,8.0,219.0,1752.0,2023-03-12 11:00:00,15
10244,16577191.0,25.0,欧美外教次卡,710.0,2023-02-28 21:21,2023-03-01 17:21:00,2023-03-31 17:21,10.0,220.0,2200.0,2023-03-31 17:21:00,30


In [114]:
official_class = official_class.merge (order_repurchase_analysis[['user_id','completed_time']], on ='user_id',how='left')
official_class = official_class.merge (class_detail,on='class_id',how='left')
official_class

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,completed_time,teacher_id,teacher_type,start_time,add_date,w_day
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,291,2,1452049200,1451871269,3
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,291,2,1452052800,1451871299,3
2,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,298,2,1452160800,1450786493,4
3,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,2016-02-17 19:27:00,342,5,1452132000,1452080945,4
4,20892,95252,131733,2,25,2016-01-07 10:21,25.0,2016-01-07 10:19,2016-03-07 10:19,2016-01-31 07:00:00,231,2,1452380400,1451933612,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487625,1363896,2926414,16450122,1,21,2023-07-27 23:06,21.0,2019-06-23 10:56,2020-02-03 10:56,2020-02-03 10:56:00,1703,1,1690635600,1689604611,6
487626,1363898,2928008,146608,1,25,2023-07-27 23:17,25.0,2016-11-07 16:01,2017-11-26 16:01,2017-11-26 16:01:00,1785,2,1690725600,1689694445,0
487627,1363899,2935812,146608,1,25,2023-07-27 23:18,25.0,2016-11-07 16:01,2017-11-26 16:01,2017-11-26 16:01:00,827,2,1690786800,1690451700,1
487628,1363902,2934702,16557469,1,25,2023-07-27 23:46,25.0,2021-12-06 19:36,2022-11-07 19:36,2022-07-07 21:00:00,738,2,1690808400,1690354675,1


In [115]:
# Convert UNIX timestamps to datetime objects
official_class['start_time'] = pd.to_datetime(official_class['start_time'], unit='s')

# Set timezone to China timezone
china_tz = pytz.timezone('Asia/Shanghai')
official_class['start_time'] = official_class['start_time'].dt.tz_localize(pytz.utc).dt.tz_convert(china_tz)

# Extract yyyy-mm-dd format from datetime objects
official_class['start_time'] = official_class['start_time'].dt.strftime('%Y-%m-%d %H:%M')

official_class

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,completed_time,teacher_id,teacher_type,start_time,add_date,w_day
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,291,2,2016-01-06 11:00,1451871269,3
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,291,2,2016-01-06 12:00,1451871299,3
2,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,298,2,2016-01-07 18:00,1450786493,4
3,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,2016-02-17 19:27:00,342,5,2016-01-07 10:00,1452080945,4
4,20892,95252,131733,2,25,2016-01-07 10:21,25.0,2016-01-07 10:19,2016-03-07 10:19,2016-01-31 07:00:00,231,2,2016-01-10 07:00,1451933612,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487625,1363896,2926414,16450122,1,21,2023-07-27 23:06,21.0,2019-06-23 10:56,2020-02-03 10:56,2020-02-03 10:56:00,1703,1,2023-07-29 21:00,1689604611,6
487626,1363898,2928008,146608,1,25,2023-07-27 23:17,25.0,2016-11-07 16:01,2017-11-26 16:01,2017-11-26 16:01:00,1785,2,2023-07-30 22:00,1689694445,0
487627,1363899,2935812,146608,1,25,2023-07-27 23:18,25.0,2016-11-07 16:01,2017-11-26 16:01,2017-11-26 16:01:00,827,2,2023-07-31 15:00,1690451700,1
487628,1363902,2934702,16557469,1,25,2023-07-27 23:46,25.0,2021-12-06 19:36,2022-11-07 19:36,2022-07-07 21:00:00,738,2,2023-07-31 21:00,1690354675,1


In [116]:
official_class = official_class[official_class['start_time']<=official_class['completed_time']]
official_class

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,completed_time,teacher_id,teacher_type,start_time,add_date,w_day
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,291,2,2016-01-06 11:00,1451871269,3
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,291,2,2016-01-06 12:00,1451871299,3
2,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,2016-05-31 15:36:00,298,2,2016-01-07 18:00,1450786493,4
3,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,2016-02-17 19:27:00,342,5,2016-01-07 10:00,1452080945,4
4,20892,95252,131733,2,25,2016-01-07 10:21,25.0,2016-01-07 10:19,2016-03-07 10:19,2016-01-31 07:00:00,231,2,2016-01-10 07:00,1451933612,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479930,1327380,2809741,16565435,2,25,2023-04-19 20:00,25.0,2022-05-08 12:12,2023-04-23 12:12,2023-04-23 12:12:00,1978,2,2023-04-22 12:00,1679225310,6
480017,1327720,2792379,16572556,2,25,2023-04-20 21:08,25.0,2022-10-25 16:31,2023-04-29 16:31,2023-04-29 16:31:00,1488,2,2023-04-22 18:00,1677379208,6
480055,1327815,2795507,16565530,2,25,2023-04-21 09:10,25.0,2022-05-27 16:51,2023-04-28 16:51,2023-04-28 16:51:00,800,2,2023-04-26 16:00,1677654661,3
480123,1328121,2831959,16570863,2,25,2023-04-22 10:25,25.0,2022-09-15 15:53,2023-04-27 15:53,2023-04-27 15:53:00,1040,5,2023-04-22 20:00,1681632035,6


In [117]:
cancel_counts = official_class[official_class['class_order_status'] == 0].groupby('user_id').size().reset_index(name='num_cancel')
order_repurchase_analysis = order_repurchase_analysis.merge(cancel_counts, on='user_id', how='left')

completed_counts = official_class[official_class['class_order_status'] == 2].groupby('user_id').size().reset_index(name='num_completed')
order_repurchase_analysis = order_repurchase_analysis.merge(completed_counts, on='user_id', how='left')

student_absent_counts = official_class[official_class['class_order_status'] == 3].groupby('user_id').size().reset_index(name='num_student_absent')
order_repurchase_analysis = order_repurchase_analysis.merge(student_absent_counts, on='user_id', how='left')

teacher_absent_counts = official_class[official_class['class_order_status'] == 4].groupby('user_id').size().reset_index(name='num_teacher_absent')
order_repurchase_analysis = order_repurchase_analysis.merge(teacher_absent_counts, on='user_id', how='left')

order_repurchase_analysis

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money,completed_time,num_days_completed,num_cancel,num_completed,num_student_absent,num_teacher_absent
0,131650.0,21.0,菲律宾外教次卡,105.0,2016-01-02 13:29,2016-01-09 10:52:00,2016-04-14 10:52,32.0,70.1,2244.0,2016-04-14 10:52:00,96,NaN,3.0,NaN,NaN
1,131669.0,25.0,欧美外教次卡,105.0,2016-01-02 13:30,2016-02-16 16:29:00,2016-07-09 16:29,48.0,149.4,7172.0,2016-07-09 16:29:00,144,16.0,8.0,NaN,NaN
2,131648.0,25.0,欧美外教次卡,105.0,2016-01-04 10:29,2016-01-06 17:22:00,2016-01-30 17:22,8.0,143.0,1144.0,2016-01-30 17:22:00,24,NaN,NaN,NaN,NaN
3,131698.0,25.0,欧美外教次卡,105.0,2016-01-05 14:32,2016-01-05 15:36:00,2016-05-31 15:36,49.0,146.0,7152.0,2016-05-31 15:36:00,147,10.0,41.0,NaN,1.0
4,131730.0,25.0,欧美外教次卡,105.0,2016-01-05 16:18,2016-01-15 07:56:00,2016-03-03 07:56,16.0,138.9,2223.0,2016-03-03 07:56:00,48,5.0,13.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16575719.0,25.0,欧美外教次卡,715.0,2023-01-30 15:12,2023-01-30 15:13:00,2023-03-31 15:13,20.0,219.0,4380.0,2023-02-24 16:00:00,25,7.0,20.0,NaN,1.0
10242,16576144.0,25.0,欧美外教次卡,710.0,2023-02-11 17:40,2023-02-14 10:20:00,2023-03-16 10:20,10.0,219.0,2190.0,2023-03-09 19:00:00,23,8.0,10.0,NaN,NaN
10243,16576983.0,25.0,欧美外教次卡,710.0,2023-02-23 13:09,2023-02-24 18:38:00,2023-04-21 18:38,8.0,219.0,1752.0,2023-03-12 11:00:00,15,NaN,8.0,NaN,NaN
10244,16577191.0,25.0,欧美外教次卡,710.0,2023-02-28 21:21,2023-03-01 17:21:00,2023-03-31 17:21,10.0,220.0,2200.0,2023-03-31 17:21:00,30,NaN,6.0,NaN,NaN


In [118]:
order_repurchase_analysis['num_cancel'] = order_repurchase_analysis['num_cancel'].fillna(0)
order_repurchase_analysis['num_completed'] = order_repurchase_analysis['num_completed'].fillna(0)
order_repurchase_analysis['num_student_absent'] = order_repurchase_analysis['num_student_absent'].fillna(0)
order_repurchase_analysis['num_teacher_absent'] = order_repurchase_analysis['num_teacher_absent'].fillna(0)
order_repurchase_analysis

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money,completed_time,num_days_completed,num_cancel,num_completed,num_student_absent,num_teacher_absent
0,131650.0,21.0,菲律宾外教次卡,105.0,2016-01-02 13:29,2016-01-09 10:52:00,2016-04-14 10:52,32.0,70.1,2244.0,2016-04-14 10:52:00,96,0.0,3.0,0.0,0.0
1,131669.0,25.0,欧美外教次卡,105.0,2016-01-02 13:30,2016-02-16 16:29:00,2016-07-09 16:29,48.0,149.4,7172.0,2016-07-09 16:29:00,144,16.0,8.0,0.0,0.0
2,131648.0,25.0,欧美外教次卡,105.0,2016-01-04 10:29,2016-01-06 17:22:00,2016-01-30 17:22,8.0,143.0,1144.0,2016-01-30 17:22:00,24,0.0,0.0,0.0,0.0
3,131698.0,25.0,欧美外教次卡,105.0,2016-01-05 14:32,2016-01-05 15:36:00,2016-05-31 15:36,49.0,146.0,7152.0,2016-05-31 15:36:00,147,10.0,41.0,0.0,1.0
4,131730.0,25.0,欧美外教次卡,105.0,2016-01-05 16:18,2016-01-15 07:56:00,2016-03-03 07:56,16.0,138.9,2223.0,2016-03-03 07:56:00,48,5.0,13.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16575719.0,25.0,欧美外教次卡,715.0,2023-01-30 15:12,2023-01-30 15:13:00,2023-03-31 15:13,20.0,219.0,4380.0,2023-02-24 16:00:00,25,7.0,20.0,0.0,1.0
10242,16576144.0,25.0,欧美外教次卡,710.0,2023-02-11 17:40,2023-02-14 10:20:00,2023-03-16 10:20,10.0,219.0,2190.0,2023-03-09 19:00:00,23,8.0,10.0,0.0,0.0
10243,16576983.0,25.0,欧美外教次卡,710.0,2023-02-23 13:09,2023-02-24 18:38:00,2023-04-21 18:38,8.0,219.0,1752.0,2023-03-12 11:00:00,15,0.0,8.0,0.0,0.0
10244,16577191.0,25.0,欧美外教次卡,710.0,2023-02-28 21:21,2023-03-01 17:21:00,2023-03-31 17:21,10.0,220.0,2200.0,2023-03-31 17:21:00,30,0.0,6.0,0.0,0.0


In [119]:
order_repurchase_analysis['per_completed'] = round(order_repurchase_analysis['num_completed'] / order_repurchase_analysis['lesson'], 2)
order_repurchase_analysis

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money,completed_time,num_days_completed,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed
0,131650.0,21.0,菲律宾外教次卡,105.0,2016-01-02 13:29,2016-01-09 10:52:00,2016-04-14 10:52,32.0,70.1,2244.0,2016-04-14 10:52:00,96,0.0,3.0,0.0,0.0,0.09
1,131669.0,25.0,欧美外教次卡,105.0,2016-01-02 13:30,2016-02-16 16:29:00,2016-07-09 16:29,48.0,149.4,7172.0,2016-07-09 16:29:00,144,16.0,8.0,0.0,0.0,0.17
2,131648.0,25.0,欧美外教次卡,105.0,2016-01-04 10:29,2016-01-06 17:22:00,2016-01-30 17:22,8.0,143.0,1144.0,2016-01-30 17:22:00,24,0.0,0.0,0.0,0.0,0.00
3,131698.0,25.0,欧美外教次卡,105.0,2016-01-05 14:32,2016-01-05 15:36:00,2016-05-31 15:36,49.0,146.0,7152.0,2016-05-31 15:36:00,147,10.0,41.0,0.0,1.0,0.84
4,131730.0,25.0,欧美外教次卡,105.0,2016-01-05 16:18,2016-01-15 07:56:00,2016-03-03 07:56,16.0,138.9,2223.0,2016-03-03 07:56:00,48,5.0,13.0,0.0,0.0,0.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16575719.0,25.0,欧美外教次卡,715.0,2023-01-30 15:12,2023-01-30 15:13:00,2023-03-31 15:13,20.0,219.0,4380.0,2023-02-24 16:00:00,25,7.0,20.0,0.0,1.0,1.00
10242,16576144.0,25.0,欧美外教次卡,710.0,2023-02-11 17:40,2023-02-14 10:20:00,2023-03-16 10:20,10.0,219.0,2190.0,2023-03-09 19:00:00,23,8.0,10.0,0.0,0.0,1.00
10243,16576983.0,25.0,欧美外教次卡,710.0,2023-02-23 13:09,2023-02-24 18:38:00,2023-04-21 18:38,8.0,219.0,1752.0,2023-03-12 11:00:00,15,0.0,8.0,0.0,0.0,1.00
10244,16577191.0,25.0,欧美外教次卡,710.0,2023-02-28 21:21,2023-03-01 17:21:00,2023-03-31 17:21,10.0,220.0,2200.0,2023-03-31 17:21:00,30,0.0,6.0,0.0,0.0,0.60


In [120]:
order_repurchase_analysis['avg_days_class']=order_repurchase_analysis['num_days_completed']/order_repurchase_analysis['num_completed']
order_repurchase_analysis

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money,completed_time,num_days_completed,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class
0,131650.0,21.0,菲律宾外教次卡,105.0,2016-01-02 13:29,2016-01-09 10:52:00,2016-04-14 10:52,32.0,70.1,2244.0,2016-04-14 10:52:00,96,0.0,3.0,0.0,0.0,0.09,32.000000
1,131669.0,25.0,欧美外教次卡,105.0,2016-01-02 13:30,2016-02-16 16:29:00,2016-07-09 16:29,48.0,149.4,7172.0,2016-07-09 16:29:00,144,16.0,8.0,0.0,0.0,0.17,18.000000
2,131648.0,25.0,欧美外教次卡,105.0,2016-01-04 10:29,2016-01-06 17:22:00,2016-01-30 17:22,8.0,143.0,1144.0,2016-01-30 17:22:00,24,0.0,0.0,0.0,0.0,0.00,inf
3,131698.0,25.0,欧美外教次卡,105.0,2016-01-05 14:32,2016-01-05 15:36:00,2016-05-31 15:36,49.0,146.0,7152.0,2016-05-31 15:36:00,147,10.0,41.0,0.0,1.0,0.84,3.585366
4,131730.0,25.0,欧美外教次卡,105.0,2016-01-05 16:18,2016-01-15 07:56:00,2016-03-03 07:56,16.0,138.9,2223.0,2016-03-03 07:56:00,48,5.0,13.0,0.0,0.0,0.81,3.692308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16575719.0,25.0,欧美外教次卡,715.0,2023-01-30 15:12,2023-01-30 15:13:00,2023-03-31 15:13,20.0,219.0,4380.0,2023-02-24 16:00:00,25,7.0,20.0,0.0,1.0,1.00,1.250000
10242,16576144.0,25.0,欧美外教次卡,710.0,2023-02-11 17:40,2023-02-14 10:20:00,2023-03-16 10:20,10.0,219.0,2190.0,2023-03-09 19:00:00,23,8.0,10.0,0.0,0.0,1.00,2.300000
10243,16576983.0,25.0,欧美外教次卡,710.0,2023-02-23 13:09,2023-02-24 18:38:00,2023-04-21 18:38,8.0,219.0,1752.0,2023-03-12 11:00:00,15,0.0,8.0,0.0,0.0,1.00,1.875000
10244,16577191.0,25.0,欧美外教次卡,710.0,2023-02-28 21:21,2023-03-01 17:21:00,2023-03-31 17:21,10.0,220.0,2200.0,2023-03-31 17:21:00,30,0.0,6.0,0.0,0.0,0.60,5.000000


In [121]:
customer_repurchase = customer_repurchase.merge(order_repurchase_analysis,on='user_id',how='left')
columns_to_drop = ['product_name', 'buy_type', 'effective_date','overdue_date','add_date','completed_time','num_days_completed']
customer_repurchase = customer_repurchase.drop(columns_to_drop, axis=1)
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf
...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286


In [122]:
num_weekday = official_class_completed[official_class_completed['w_day'].isin([1, 2, 3, 4, 5])].groupby('user_id')['w_day'].count().reset_index()
num_weekday.columns = ['user_id', 'num_weekday']

customer_repurchase = customer_repurchase.merge(num_weekday, on='user_id', how='left')
customer_repurchase['num_weekday']=customer_repurchase['num_weekday'].fillna(0)

num_weekend = official_class_completed[official_class_completed['w_day'].isin([6, 0])].groupby('user_id')['w_day'].count().reset_index()
num_weekend.columns = ['user_id', 'num_weekend']

customer_repurchase = customer_repurchase.merge(num_weekend, on='user_id', how='left')
customer_repurchase['num_weekend']=customer_repurchase['num_weekend'].fillna(0)
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,num_weekday,num_weekend
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,54.0,21.0
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,17.0,10.0
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,4.0,0.0
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286,7.0,0.0


In [123]:
customer_repurchase['per_weekday']=customer_repurchase['num_weekday']/customer_repurchase['num_completed']
customer_repurchase['per_weekend']=customer_repurchase['num_weekend']/customer_repurchase['num_completed']

customer_repurchase.drop('num_weekday',axis=1,inplace=True)
customer_repurchase.drop('num_weekend',axis=1,inplace=True)

customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,per_weekday,per_weekend
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,0.72000,0.28000
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,0.62963,0.37037
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,1.00000,0.00000
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286,1.00000,0.00000


In [124]:
fav_teacher = official_class_completed.groupby('user_id')['teacher_id'].agg(lambda x: x.value_counts().idxmax()).reset_index()

customer_repurchase = customer_repurchase.merge(fav_teacher, on='user_id', how='left')
customer_repurchase['teacher_id']=customer_repurchase['teacher_id'].fillna(0)
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,per_weekday,per_weekend,teacher_id
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,0.72000,0.28000,825.0
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,0.62963,0.37037,571.0
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,1.00000,0.00000,1875.0
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286,1.00000,0.00000,1465.0


In [125]:
num_teacher = official_class_completed.groupby('user_id')['teacher_id'].nunique().reset_index(name='num_teacher')

customer_repurchase = customer_repurchase.merge(num_teacher, on='user_id', how='left')
customer_repurchase['num_teacher']=customer_repurchase['num_teacher'].fillna(0)
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,per_weekday,per_weekend,teacher_id,num_teacher
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,0.72000,0.28000,825.0,12.0
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,0.62963,0.37037,571.0,4.0
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,1.00000,0.00000,1875.0,2.0
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286,1.00000,0.00000,1465.0,5.0


In [126]:
class_feedback_auto = class_feedback_auto.drop('issue_type',axis=1)
class_feedback.drop(['issue_type_1', 'issue_type_2','issue_content'], axis=1, inplace=True)

class_feedback = pd.concat([class_feedback_auto, class_feedback], ignore_index=True)
class_feedback

,class_order_id
0,1329397
1,1328083
2,1329255
3,1327062
4,1327039
...,...
34264,1360868
34265,1358795
34266,1362043
34267,1360403


In [127]:
class_feedback = class_feedback.drop_duplicates()
class_feedback

,class_order_id
0,1329397
1,1328083
2,1329255
3,1327062
4,1327039
...,...
34262,1361600
34263,1360926
34264,1360868
34265,1358795


In [128]:
official_class_completed['has_issue'] = np.where(official_class_completed['class_order_id'].isin(class_feedback['class_order_id']), 1, 0)
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,teacher_type,start_time,add_date,w_day,row_rank,lesson,completed_time,has_issue
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 11:00,1451871269,3,1,49.0,NaN,0
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 12:00,1451871299,3,2,49.0,NaN,0
2,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 10:00,1452080945,4,1,14.0,NaN,0
3,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,2,2016-01-07 18:00,1450786493,4,3,49.0,NaN,0
4,20935,95052,131732,2,32,2016-01-07 17:30,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 21:00,1451897903,4,2,14.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210929,1326792,2830267,16565435,2,25,2023-04-18 09:56,25.0,2022-05-08 12:12,2023-04-23 12:12,1898,2,2023-04-22 20:00,1681478224,6,48,50.0,NaN,0
210930,1328121,2831959,16570863,2,25,2023-04-22 10:25,25.0,2022-09-15 15:53,2023-04-27 15:53,1040,5,2023-04-22 20:00,1681632035,6,21,32.0,NaN,0
210931,1326632,2826472,16566901,2,23,2023-04-17 19:32,23.0,2022-06-14 10:42,2023-04-25 10:42,1419,5,2023-04-22 21:00,1681088884,6,105,105.0,2023-04-22 21:00,0
210932,1327815,2795507,16565530,2,25,2023-04-21 09:10,25.0,2022-05-27 16:51,2023-04-28 16:51,800,2,2023-04-26 16:00,1677654661,3,29,48.0,NaN,0


In [129]:
num_issue = official_class_completed.groupby('user_id')['has_issue'].sum().reset_index(name='num_issue')
customer_repurchase = customer_repurchase.merge(num_issue, on='user_id', how='left')
customer_repurchase['num_issue']=customer_repurchase['num_issue'].fillna(0)
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,per_weekday,per_weekend,teacher_id,num_teacher,num_issue
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,0.72000,0.28000,825.0,12.0,0.0
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,0.62963,0.37037,571.0,4.0,0.0
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,1.00000,0.00000,1875.0,2.0,0.0
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286,1.00000,0.00000,1465.0,5.0,0.0


In [130]:
official_class_completed = official_class_completed.merge(class_comment,on='class_order_id',how='left')
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,teacher_type,start_time,add_date,w_day,row_rank,lesson,completed_time,has_issue,total_point,material
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 11:00,1451871269,3,1,49.0,NaN,0,5,4
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 12:00,1451871299,3,2,49.0,NaN,0,5,4
2,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 10:00,1452080945,4,1,14.0,NaN,0,NaN,NaN
3,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,2,2016-01-07 18:00,1450786493,4,3,49.0,NaN,0,4,3
4,20935,95052,131732,2,32,2016-01-07 17:30,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 21:00,1451897903,4,2,14.0,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204852,1326792,2830267,16565435,2,25,2023-04-18 09:56,25.0,2022-05-08 12:12,2023-04-23 12:12,1898,2,2023-04-22 20:00,1681478224,6,48,50.0,NaN,0,NaN,NaN
204853,1328121,2831959,16570863,2,25,2023-04-22 10:25,25.0,2022-09-15 15:53,2023-04-27 15:53,1040,5,2023-04-22 20:00,1681632035,6,21,32.0,NaN,0,NaN,NaN
204854,1326632,2826472,16566901,2,23,2023-04-17 19:32,23.0,2022-06-14 10:42,2023-04-25 10:42,1419,5,2023-04-22 21:00,1681088884,6,105,105.0,2023-04-22 21:00,0,NaN,NaN
204855,1327815,2795507,16565530,2,25,2023-04-21 09:10,25.0,2022-05-27 16:51,2023-04-28 16:51,800,2,2023-04-26 16:00,1677654661,3,29,48.0,NaN,0,NaN,NaN


In [131]:
official_class_completed['total_point'] = pd.to_numeric(official_class_completed['total_point'], errors='coerce')
avg_comment_total = official_class_completed.groupby('user_id')['total_point'].mean().reset_index()
customer_repurchase = customer_repurchase.merge(avg_comment_total,on='user_id', how='left')

official_class_completed['material'] = pd.to_numeric(official_class_completed['material'], errors='coerce')
avg_comment_material = official_class_completed.groupby('user_id')['material'].mean().reset_index()
customer_repurchase = customer_repurchase.merge(avg_comment_material,on='user_id', how='left')

customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,per_weekday,per_weekend,teacher_id,num_teacher,num_issue,total_point,material
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,NaN,NaN
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,0.72000,0.28000,825.0,12.0,0.0,NaN,NaN
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,NaN,NaN
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,NaN,NaN
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,NaN,NaN
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,0.62963,0.37037,571.0,4.0,0.0,NaN,NaN
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,1.00000,0.00000,1875.0,2.0,0.0,NaN,NaN
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286,1.00000,0.00000,1465.0,5.0,0.0,NaN,NaN


In [132]:
customer_repurchase['total_point'].replace(0, np.nan, inplace=True)
customer_repurchase['material'].replace(0, np.nan, inplace=True)

In [133]:
customer_repurchase['total_point'].info()
customer_repurchase['material'].info()

<class 'pandas.core.series.Series'>
Int64Index: 10246 entries, 0 to 10245
Series name: total_point
Non-Null Count  Dtype  
--------------  -----  
3632 non-null   float64
dtypes: float64(1)
memory usage: 160.1 KB
<class 'pandas.core.series.Series'>
Int64Index: 10246 entries, 0 to 10245
Series name: material
Non-Null Count  Dtype  
--------------  -----  
3632 non-null   float64
dtypes: float64(1)
memory usage: 160.1 KB


In [134]:
total_point_mean = round(customer_repurchase['total_point'].mean(), 2)
customer_repurchase['total_point'].replace(np.nan, total_point_mean, inplace=True)

material_mean = round(customer_repurchase['material'].mean(), 2)
customer_repurchase['material'].replace(np.nan, material_mean, inplace=True)
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,per_weekday,per_weekend,teacher_id,num_teacher,num_issue,total_point,material
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,0.72000,0.28000,825.0,12.0,0.0,4.72,4.7
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,0.62963,0.37037,571.0,4.0,0.0,4.72,4.7
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,1.00000,0.00000,1875.0,2.0,0.0,4.72,4.7
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286,1.00000,0.00000,1465.0,5.0,0.0,4.72,4.7


In [135]:
official_class_completed['add_date'] = pd.to_datetime(official_class_completed['add_date'], unit='s')

china_tz = pytz.timezone('Asia/Shanghai')
official_class_completed['add_date'] = official_class_completed['add_date'].dt.tz_localize(pytz.utc).dt.tz_convert(china_tz)

official_class_completed['add_date'] = official_class_completed['add_date'].dt.strftime('%Y-%m-%d %H:%M')

official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,teacher_type,start_time,add_date,w_day,row_rank,lesson,completed_time,has_issue,total_point,material
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 11:00,2016-01-04 09:34,3,1,49.0,NaN,0,5.0,4.0
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,2,2016-01-06 12:00,2016-01-04 09:34,3,2,49.0,NaN,0,5.0,4.0
2,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 10:00,2016-01-06 19:49,4,1,14.0,NaN,0,NaN,NaN
3,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,2,2016-01-07 18:00,2015-12-22 20:14,4,3,49.0,NaN,0,4.0,3.0
4,20935,95052,131732,2,32,2016-01-07 17:30,32.0,2016-01-06 19:27,2016-02-17 19:27,342,5,2016-01-07 21:00,2016-01-04 16:58,4,2,14.0,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204852,1326792,2830267,16565435,2,25,2023-04-18 09:56,25.0,2022-05-08 12:12,2023-04-23 12:12,1898,2,2023-04-22 20:00,2023-04-14 21:17,6,48,50.0,NaN,0,NaN,NaN
204853,1328121,2831959,16570863,2,25,2023-04-22 10:25,25.0,2022-09-15 15:53,2023-04-27 15:53,1040,5,2023-04-22 20:00,2023-04-16 16:00,6,21,32.0,NaN,0,NaN,NaN
204854,1326632,2826472,16566901,2,23,2023-04-17 19:32,23.0,2022-06-14 10:42,2023-04-25 10:42,1419,5,2023-04-22 21:00,2023-04-10 09:08,6,105,105.0,2023-04-22 21:00,0,NaN,NaN
204855,1327815,2795507,16565530,2,25,2023-04-21 09:10,25.0,2022-05-27 16:51,2023-04-28 16:51,800,2,2023-04-26 16:00,2023-03-01 15:11,3,29,48.0,NaN,0,NaN,NaN


In [136]:
official_class_completed['start_time'] = pd.to_datetime(official_class_completed['start_time'])
official_class_completed['add_date'] = pd.to_datetime(official_class_completed['add_date'])
official_class_completed['num_days_reserve'] = (official_class_completed['start_time'] - official_class_completed['add_date']).dt.days
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,...,start_time,add_date,w_day,row_rank,lesson,completed_time,has_issue,total_point,material,num_days_reserve
0,20737,94793,131698,2,25,2016-01-05 15:50,25.0,2016-01-05 15:36,2016-05-31 15:36,291,...,2016-01-06 11:00:00,2016-01-04 09:34:00,3,1,49.0,NaN,0,5.0,4.0,2
1,20739,94794,131698,2,25,2016-01-05 15:55,25.0,2016-01-05 15:36,2016-05-31 15:36,291,...,2016-01-06 12:00:00,2016-01-04 09:34:00,3,2,49.0,NaN,0,5.0,4.0,2
2,20868,95880,131732,2,32,2016-01-06 20:03,32.0,2016-01-06 19:27,2016-02-17 19:27,342,...,2016-01-07 10:00:00,2016-01-06 19:49:00,4,1,14.0,NaN,0,NaN,NaN,0
3,20744,90326,131698,2,25,2016-01-05 16:24,25.0,2016-01-05 15:36,2016-05-31 15:36,298,...,2016-01-07 18:00:00,2015-12-22 20:14:00,4,3,49.0,NaN,0,4.0,3.0,15
4,20935,95052,131732,2,32,2016-01-07 17:30,32.0,2016-01-06 19:27,2016-02-17 19:27,342,...,2016-01-07 21:00:00,2016-01-04 16:58:00,4,2,14.0,NaN,0,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204852,1326792,2830267,16565435,2,25,2023-04-18 09:56,25.0,2022-05-08 12:12,2023-04-23 12:12,1898,...,2023-04-22 20:00:00,2023-04-14 21:17:00,6,48,50.0,NaN,0,NaN,NaN,7
204853,1328121,2831959,16570863,2,25,2023-04-22 10:25,25.0,2022-09-15 15:53,2023-04-27 15:53,1040,...,2023-04-22 20:00:00,2023-04-16 16:00:00,6,21,32.0,NaN,0,NaN,NaN,6
204854,1326632,2826472,16566901,2,23,2023-04-17 19:32,23.0,2022-06-14 10:42,2023-04-25 10:42,1419,...,2023-04-22 21:00:00,2023-04-10 09:08:00,6,105,105.0,2023-04-22 21:00,0,NaN,NaN,12
204855,1327815,2795507,16565530,2,25,2023-04-21 09:10,25.0,2022-05-27 16:51,2023-04-28 16:51,800,...,2023-04-26 16:00:00,2023-03-01 15:11:00,3,29,48.0,NaN,0,NaN,NaN,56


In [137]:
average_reserve_days = official_class_completed.groupby('user_id')['num_days_reserve'].mean().reset_index(name='avg_days_reserve')
customer_repurchase = customer_repurchase.merge(average_reserve_days,on='user_id', how='left')
customer_repurchase['avg_days_reserve']=customer_repurchase['avg_days_reserve'].fillna(0)
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,per_weekday,per_weekend,teacher_id,num_teacher,num_issue,total_point,material,avg_days_reserve
0,133809,99.0,21.0,32.0,59.0,1888.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7,0.000000
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,0.72000,0.28000,825.0,12.0,0.0,4.72,4.7,5.706667
2,16485247,99.0,25.0,32.0,174.1,5572.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7,0.000000
3,16531907,99.0,21.0,103.0,55.0,5664.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7,0.000000
4,148917,94.0,25.0,5.0,160.0,800.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16528592,99.0,21.0,50.0,65.0,3250.0,0.0,0.0,0.0,0.0,0.00,inf,NaN,NaN,0.0,0.0,0.0,4.72,4.7,0.000000
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,0.62963,0.37037,571.0,4.0,0.0,4.72,4.7,8.629630
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,1.00000,0.00000,1875.0,2.0,0.0,4.72,4.7,8.000000
10244,16501431,0.0,21.0,32.0,69.0,2208.0,1.0,7.0,0.0,0.0,0.22,13.714286,1.00000,0.00000,1465.0,5.0,0.0,4.72,4.7,4.285714


In [138]:
customer_repurchase = customer_repurchase[customer_repurchase['num_completed']!=0]
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,per_completed,avg_days_class,per_weekday,per_weekend,teacher_id,num_teacher,num_issue,total_point,material,avg_days_reserve
1,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,0.62,4.840000,0.720000,0.280000,825.0,12.0,0.0,4.72,4.7,5.706667
5,136529,97.0,25.0,48.0,145.8,7000.0,0.0,1.0,0.0,0.0,0.02,144.000000,0.000000,1.000000,426.0,1.0,0.0,4.72,4.7,7.000000
6,132759,99.0,25.0,32.0,149.0,4767.0,1.0,11.0,0.0,0.0,0.34,8.727273,0.909091,0.090909,239.0,4.0,0.0,4.72,4.7,5.454545
7,132826,99.0,25.0,32.0,149.0,4768.0,3.0,2.0,1.0,0.0,0.06,48.000000,0.500000,0.500000,253.0,1.0,0.0,4.72,4.7,8.500000
8,132151,99.0,23.0,10.0,96.0,960.0,1.0,2.0,0.0,0.0,0.20,15.000000,1.000000,0.000000,351.0,1.0,0.0,4.72,4.7,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10239,16445436,99.0,25.0,32.0,187.4,5998.0,4.0,4.0,0.0,0.0,0.12,24.000000,1.000000,0.000000,784.0,4.0,0.0,4.72,4.7,27.750000
10240,16468966,97.0,25.0,36.0,159.2,5731.0,3.0,14.0,1.0,0.0,0.39,7.714286,0.571429,0.428571,291.0,3.0,0.0,4.72,4.7,12.642857
10242,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,0.84,8.296296,0.629630,0.370370,571.0,4.0,0.0,4.72,4.7,8.629630
10243,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,0.40,17.500000,1.000000,0.000000,1875.0,2.0,0.0,4.72,4.7,8.000000


In [139]:
official_class_completed['effective_date'] = pd.to_datetime(official_class_completed['effective_date'])


# Sort the DataFrame by user_id and row_rank
official_class_completed = official_class_completed.sort_values(['user_id', 'row_rank'])

# Calculate the time difference between rows
official_class_completed['num_days_class'] = official_class_completed['start_time'] - official_class_completed.groupby('user_id')['start_time'].shift(1)

# For rows with row_rank 1, calculate the time difference with effective_time
official_class_completed.loc[official_class_completed['row_rank'] == 1, 'num_days_class'] = official_class_completed['start_time'] - official_class_completed['effective_date']


# Convert the timedelta column to days
official_class_completed['num_days_class'] = official_class_completed['num_days_class'].dt.days

# Reset the DataFrame index
official_class_completed = official_class_completed.reset_index(drop=True)

# Display the updated DataFrame
official_class_completed

,class_order_id,class_id,user_id,class_order_status,pay_type,order_time,product_id,effective_date,overdue_date,teacher_id,...,add_date,w_day,row_rank,lesson,completed_time,has_issue,total_point,material,num_days_reserve,num_days_class
0,24655,112271,131647,2,25,2016-02-16 09:55,25.0,2016-01-22 14:08:00,2016-11-05 14:08,170,...,2016-02-14 00:38:00,2,1,96.0,NaN,0,NaN,NaN,2,25
1,24947,111728,131647,2,25,2016-02-18 12:47,25.0,2016-01-22 14:08:00,2016-11-05 14:08,231,...,2016-02-12 22:41:00,5,2,96.0,NaN,0,NaN,NaN,7,3
2,26150,116684,131647,2,25,2016-02-29 08:53,25.0,2016-01-22 14:08:00,2016-11-05 14:08,255,...,2016-02-23 18:48:00,1,3,96.0,NaN,0,NaN,NaN,6,10
3,29548,121976,131647,2,25,2016-03-23 09:13,25.0,2016-01-22 14:08:00,2016-11-05 14:08,203,...,2016-03-06 07:46:00,3,4,96.0,NaN,0,NaN,NaN,17,22
4,30282,123092,131647,2,25,2016-03-28 09:41,25.0,2016-01-22 14:08:00,2016-11-05 14:08,226,...,2016-03-07 23:07:00,3,5,96.0,NaN,0,NaN,NaN,22,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204852,1317094,2812115,16577191,2,25,2023-03-21 18:42,25.0,2023-03-01 17:21:00,2023-03-31 17:21,1410,...,2023-03-21 18:40:00,3,6,10.0,NaN,0,NaN,NaN,1,5
204853,1313620,2804619,16577836,2,25,2023-03-12 14:01,25.0,2023-03-12 13:42:00,2023-04-29 13:42,1893,...,2023-03-11 17:53:00,0,1,16.0,NaN,0,NaN,NaN,1,0
204854,1313806,2800125,16577836,2,25,2023-03-12 23:54,25.0,2023-03-12 13:42:00,2023-04-29 13:42,1893,...,2023-03-06 16:01:00,1,2,16.0,NaN,0,5.0,5.0,6,0
204855,1314114,2805431,16577836,2,25,2023-03-13 17:09,25.0,2023-03-12 13:42:00,2023-04-29 13:42,1893,...,2023-03-12 19:25:00,1,3,16.0,NaN,0,NaN,NaN,1,0


In [140]:
# Calculate the standard deviation of num_days_class for each user
std_days_class = official_class_completed.groupby('user_id')['num_days_class'].std().reset_index(name='std_days_between_class')

customer_repurchase = customer_repurchase.merge(std_days_class,on='user_id', how='left')

customer_repurchase['std_days_between_class'] = customer_repurchase['std_days_between_class'].fillna(0)

customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,...,avg_days_class,per_weekday,per_weekend,teacher_id,num_teacher,num_issue,total_point,material,avg_days_reserve,std_days_between_class
0,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,...,4.840000,0.720000,0.280000,825.0,12.0,0.0,4.72,4.7,5.706667,12.285009
1,136529,97.0,25.0,48.0,145.8,7000.0,0.0,1.0,0.0,0.0,...,144.000000,0.000000,1.000000,426.0,1.0,0.0,4.72,4.7,7.000000,0.000000
2,132759,99.0,25.0,32.0,149.0,4767.0,1.0,11.0,0.0,0.0,...,8.727273,0.909091,0.090909,239.0,4.0,0.0,4.72,4.7,5.454545,5.356390
3,132826,99.0,25.0,32.0,149.0,4768.0,3.0,2.0,1.0,0.0,...,48.000000,0.500000,0.500000,253.0,1.0,0.0,4.72,4.7,8.500000,13.435029
4,132151,99.0,23.0,10.0,96.0,960.0,1.0,2.0,0.0,0.0,...,15.000000,1.000000,0.000000,351.0,1.0,0.0,4.72,4.7,3.000000,0.707107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9591,16445436,99.0,25.0,32.0,187.4,5998.0,4.0,4.0,0.0,0.0,...,24.000000,1.000000,0.000000,784.0,4.0,0.0,4.72,4.7,27.750000,2.160247
9592,16468966,97.0,25.0,36.0,159.2,5731.0,3.0,14.0,1.0,0.0,...,7.714286,0.571429,0.428571,291.0,3.0,0.0,4.72,4.7,12.642857,19.112880
9593,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,...,8.296296,0.629630,0.370370,571.0,4.0,0.0,4.72,4.7,8.629630,6.189321
9594,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,...,17.500000,1.000000,0.000000,1875.0,2.0,0.0,4.72,4.7,8.000000,9.327379


In [141]:
customer_repurchase = customer_repurchase.merge(cr,on='user_id',how='left')
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,...,per_weekday,per_weekend,teacher_id,num_teacher,num_issue,total_point,material,avg_days_reserve,std_days_between_class,cr_id
0,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,...,0.720000,0.280000,825.0,12.0,0.0,4.72,4.7,5.706667,12.285009,0.0
1,136529,97.0,25.0,48.0,145.8,7000.0,0.0,1.0,0.0,0.0,...,0.000000,1.000000,426.0,1.0,0.0,4.72,4.7,7.000000,0.000000,233.0
2,132759,99.0,25.0,32.0,149.0,4767.0,1.0,11.0,0.0,0.0,...,0.909091,0.090909,239.0,4.0,0.0,4.72,4.7,5.454545,5.356390,233.0
3,132826,99.0,25.0,32.0,149.0,4768.0,3.0,2.0,1.0,0.0,...,0.500000,0.500000,253.0,1.0,0.0,4.72,4.7,8.500000,13.435029,233.0
4,132151,99.0,23.0,10.0,96.0,960.0,1.0,2.0,0.0,0.0,...,1.000000,0.000000,351.0,1.0,0.0,4.72,4.7,3.000000,0.707107,233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9591,16445436,99.0,25.0,32.0,187.4,5998.0,4.0,4.0,0.0,0.0,...,1.000000,0.000000,784.0,4.0,0.0,4.72,4.7,27.750000,2.160247,651.0
9592,16468966,97.0,25.0,36.0,159.2,5731.0,3.0,14.0,1.0,0.0,...,0.571429,0.428571,291.0,3.0,0.0,4.72,4.7,12.642857,19.112880,667.0
9593,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,...,0.629630,0.370370,571.0,4.0,0.0,4.72,4.7,8.629630,6.189321,154.0
9594,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,...,1.000000,0.000000,1875.0,2.0,0.0,4.72,4.7,8.000000,9.327379,319.0


In [142]:
repurchase_type = attribute_buy_type[attribute_buy_type['attri_name'].isin(['续费充值','续费'])]

In [143]:
order_repurchase = official_order[official_order['buy_type'].isin(repurchase_type['attri_id'])]
order_repurchase

,user_id,product_id,product_name,buy_type,add_date,effective_date,overdue_date,lesson,unit_price,product_money
1748,114733.0,21.0,菲律宾外教,107.0,1.437724e+09,1.437881e+09,1.443929e+09,10.0,49.0,490.0
1794,118031.0,23.0,专业中教,107.0,1.437982e+09,1.437987e+09,1.440406e+09,4.0,119.0,476.0
1834,119018.0,21.0,菲律宾外教次卡,107.0,1.438138e+09,1.438170e+09,1.442318e+09,16.0,49.0,784.0
1853,119029.0,25.0,欧美外教次卡,107.0,1.438231e+09,NaN,NaN,1.0,169.0,169.0
1865,117276.0,25.0,欧美外教次卡,107.0,1.438244e+09,1.440403e+09,1.444550e+09,16.0,118.2,1892.0
...,...,...,...,...,...,...,...,...,...,...
230534,16571023.0,25.0,欧美外教次卡,713.0,1.689927e+09,1.689927e+09,1.695111e+09,20.0,219.0,4380.0
230766,16573652.0,25.0,欧美外教次卡,713.0,1.690178e+09,NaN,NaN,10.0,220.0,2200.0
230777,16446849.0,25.0,欧美外教次卡,713.0,1.690182e+09,NaN,NaN,51.0,182.2,9291.0
230794,16577697.0,25.0,欧美外教次卡,713.0,1.690186e+09,NaN,NaN,62.0,179.2,11111.0


In [144]:
repurchase_user = order_repurchase['user_id'].unique().tolist()
repurchase_user.extend(user_with_multiple_new_orders)

customer_repurchase['if_repurchase'] = customer_repurchase['user_id'].apply(lambda x: 1 if x in repurchase_user else 0)
customer_repurchase

,user_id,channel,product_id,lesson,unit_price,product_money,num_cancel,num_completed,num_student_absent,num_teacher_absent,...,per_weekend,teacher_id,num_teacher,num_issue,total_point,material,avg_days_reserve,std_days_between_class,cr_id,if_repurchase
0,168116,97.0,21.0,121.0,41.3,5000.0,56.0,75.0,28.0,2.0,...,0.280000,825.0,12.0,0.0,4.72,4.7,5.706667,12.285009,0.0,0
1,136529,97.0,25.0,48.0,145.8,7000.0,0.0,1.0,0.0,0.0,...,1.000000,426.0,1.0,0.0,4.72,4.7,7.000000,0.000000,233.0,0
2,132759,99.0,25.0,32.0,149.0,4767.0,1.0,11.0,0.0,0.0,...,0.090909,239.0,4.0,0.0,4.72,4.7,5.454545,5.356390,233.0,0
3,132826,99.0,25.0,32.0,149.0,4768.0,3.0,2.0,1.0,0.0,...,0.500000,253.0,1.0,0.0,4.72,4.7,8.500000,13.435029,233.0,0
4,132151,99.0,23.0,10.0,96.0,960.0,1.0,2.0,0.0,0.0,...,0.000000,351.0,1.0,0.0,4.72,4.7,3.000000,0.707107,233.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9591,16445436,99.0,25.0,32.0,187.4,5998.0,4.0,4.0,0.0,0.0,...,0.000000,784.0,4.0,0.0,4.72,4.7,27.750000,2.160247,651.0,0
9592,16468966,97.0,25.0,36.0,159.2,5731.0,3.0,14.0,1.0,0.0,...,0.428571,291.0,3.0,0.0,4.72,4.7,12.642857,19.112880,667.0,0
9593,16554336,99.0,25.0,32.0,199.0,6368.0,10.0,27.0,0.0,0.0,...,0.370370,571.0,4.0,0.0,4.72,4.7,8.629630,6.189321,154.0,0
9594,16564800,99.0,25.0,10.0,209.0,2090.0,1.0,4.0,0.0,0.0,...,0.000000,1875.0,2.0,0.0,4.72,4.7,8.000000,9.327379,319.0,0


# Finalization

In [145]:
customer_repurchase['if_repurchase'].value_counts()

0    6796
1    2800
Name: if_repurchase, dtype: int64

In [146]:
customer_repurchase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9596 entries, 0 to 9595
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 9596 non-null   int64  
 1   channel                 9596 non-null   float64
 2   product_id              9596 non-null   float64
 3   lesson                  9596 non-null   float64
 4   unit_price              9596 non-null   float64
 5   product_money           9596 non-null   float64
 6   num_cancel              9596 non-null   float64
 7   num_completed           9596 non-null   float64
 8   num_student_absent      9596 non-null   float64
 9   num_teacher_absent      9596 non-null   float64
 10  per_completed           9596 non-null   float64
 11  avg_days_class          9596 non-null   float64
 12  per_weekday             9596 non-null   float64
 13  per_weekend             9596 non-null   float64
 14  teacher_id              9596 non-null   

In [147]:
# Save the DataFrame to a CSV file
customer_repurchase.to_csv('customer_repurchase.csv', index=False)

print("saved to CSV file successfully.")

saved to CSV file successfully.
